<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорты-необходимых-библиотек" data-toc-modified-id="Импорты-необходимых-библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорты необходимых библиотек</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li></ul></div>

# Защита персональных данных клиентов

Цель проекта - защитить данные клиентов страховой компании «Хоть потоп». Для этого нужно разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Необходимо привести обоснование корректности его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Задачи:**
* загрузить и изучить представленные данные.
* Ответить на вопрос, изменится ли качество линейной регрессии, если умножить матрицу признаков на обратимую матрицу.
* Предложить и обосновать алгоритм преобразования данных, который позволит прогнозировать целевой признак также эффективно, как на исходных данных.
* Запрограммировать алгоритм и проверить качество линейной регрессии на исходных и преобразованных данных с помощью метрики R2.

## Импорты необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score

## Загрузка данных

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
display(df.head())

df.info()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
# посмотрим, есть ли повторы в данных
print(df.duplicated().sum())


153


In [4]:
# сразу удалим повторы
df = df.drop_duplicates()

# и посмотрим, как распределены данные
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,4847.000000,4847.000000,4847.000000,4847.000000,4847.000000
mean,0.498453,31.023932,39895.811842,1.203425,0.152259
std,0.500049,8.487995,9972.953985,1.098664,0.468934
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33200.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
# посмотрим, отличаются ли чем-то клиенты, которые хотя бы однажды получали выплаты
df[df['Страховые выплаты'] > 0].describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,563.000000,563.000000,563.000000,563.000000,563.000000
mean,0.497336,46.536412,39877.264654,1.103020,1.310835
std,0.500438,3.889031,10073.880905,1.040142,0.612160
min,0.000000,42.000000,11200.000000,0.000000,1.000000
25%,0.000000,44.000000,32800.000000,0.000000,1.000000
50%,0.000000,46.000000,40200.000000,1.000000,1.000000
75%,1.000000,49.000000,46700.000000,2.000000,1.000000
max,1.000000,65.000000,70500.000000,6.000000,5.000000


Итак, в данных содержатся следующие признаки:
* пол клиента, обозначаемый как "1" или "0". Видим, что клиентов с полом "0" существенно больше.
* возраст клиентов - от 18 до 65 лет.
* зарплата клиентов - от 5300 до 7900, довольно большой разброс.
* количество членов семьи клиента (по всей видимости, кроме него самого), которое составляет от 0 до 6. 

Целевой признак - количество раз, когда клиент получал страховые выплаты. Большинство клиентов их не получали, однако некоторые - получали до 5 раз. Похоже, что клиенты, которые получали выплаты, в среднем старше, однако нельзя сказать, что их доходы и другие признаки заметно отличаются от выборки в целом.

Поскольку типы данных соответствуют задаче исследования и пропусков в датасете не обнаружено, подготовка данных свелась к поиску и удалению дубликатов. 

В рамках данного исследования нет задачи улучшить качество предсказаний модели и выявить особенности в данных, поэтому отказываемся от выявления коррелляций между признаками и подробного анализа их распределения.

## Умножение матриц

Нам необходимо ответить на вопрос, изменится ли качество линейной регрессии, если умножить признаки на обратимую матрицу. 

**Ответ:** Качество модели не изменится. Это следует из формулы обучения линейной регрессии.

**Обоснование:**
Возьмём следующие обозначения.

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки (должна быть обратимой)

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Вектор предсказаний задаётся формулой $a = Xw$. 

Формула обучения линейной регрессии: $w = (X^T X)^{-1} X^T y$

Преобразуем выражение для вектора весов линейной регрессии $w'$. Примем, что при умножении исходной матрицы $X$ на обратимую матрицу $P$ у нас получается некоторая матрица $Y$:
$$
w' = (Y^TY)^{-1} Y^Ty
$$

Тогда:
$$
w' = ((XP)^T(XP))^{-1} (XP)^Ty = P^{-1}((XP)^{T}X)^{-1} (XP)^Ty =
P^{-1}((XP)^{T}X)^{-1} P^TX^Ty = P^{-1}(P^TX^TX)^{-1} P^TX^Ty =
P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^Ty
$$

Сократим получившееся выражение: $(P^{T})^{-1} P^T = P^{-1}(X^TX)^{-1} X^Ty$

Произведём замену для $w$: $w = (X^TX)^{-1}X^Ty = P^{-1}w$

Тогда: $w(y)= P^{-1}w$

Теперь посмотрим, что получается в формуле расчёта $a'$:
$$
a'=X'w' = XPP^{-1}w = Xw
$$
То есть, получили выражение, тождественное $a$. Таким образом, векторы предсказаний $w$ и $w'$ не отличаются.

## Алгоритм преобразования

В соответствии с задачей, необходимо предложить такой алгоритм преобразования данных, чтобы по ним было сложно восстановить исходную информацию о клиентах, однако чтобы качество линейной регрессии не ухудшилось.

На основании приведённого выше теоретического обоснования можно предложить следующий **алгоритм**:
1) отделяем от набора данных целевой признак.

2) проводим обучение модели на исходных (`source`) данных, рассчитываем метрику R2.

3) генерируем случайную матрицу (`key_matrix`), которая будет ключом шифрования, и сохраняем её.

4) проверяем случайную матрицу на обратимость.

5) умножаем набор данных на ключ шифрования, получаем преобразованный набор.

6) обучаем модель на преобразованных данных, находим вектор предсказаний, вычисляем метрику R2.

7) сравниваем получившиеся метрики.

*Предположим, что после этого нам потребуется восстановить исходный набор данных. Внесём этот шаг в алгоритм - чтобы проверить успешность преобразования.*

8) дешифруем исходные данные, умножая преобразованный набор на обратную матрицу для ключа шифрования.

**Обоснование**

Смысл перемножения двух матриц состоит в том, что все векторы-строки матрицы X скалярно умножаются на все векторы-столбцы матрицы Y. Если мы перемножим каждую строку (набор признаков) матрицы Х на одинаковые наборы векторов (каждый столбец) матрицы Y, то на выходе получим новую матрицу - набор векторов, в котором расстояния между ними будут такими же, как и в исходной матрице. 

## Проверка алгоритма

Необходимо проверить, будет ли отличаться теоретически предсказанный эффект от реального. Для этого запрограммируем алгоритм и рассчитаем метрику R2 на преобразованном наборе данных.

Предварительно, однако, нужно рассчитать эту же метрику на **исходном наборе данных** (дадим ему наименование `source`).

In [6]:
# отделим целевой признак от остальных
features_source = df.drop('Страховые выплаты', axis=1)
target_source = df['Страховые выплаты']

features_source.shape

(4847, 4)

In [7]:
# обучим модель линейной регрессии
model_source = LinearRegression()
model_source.fit(features_source, target_source)

# получим предсказания и рассчитаем метрику R2
predictions_source = model_source.predict(features_source)
print(r2_score(target_source, predictions_source))

0.4302010044852066


Теперь создадим функцию, которая будет преобразовывать исходный набор признаков и получать метрику R2 на нём. Внутри функции подробно распишем этапы алгоритма, а также будем выводить промежуточные итоги, чтобы убедиться, что всё работает как должно.

In [8]:
def encryption(data, column_name):
    # отделяем признаки
    features = data.drop(column_name, axis=1)
    target = data[column_name]
    
    #обучаем модель на исходных данных и получаем метрику
    model_source = LinearRegression().fit(features, target)
    predictions_source = model_source.predict(features)
    r2_source = r2_score(target, predictions_source)
    print('Метрика R2 на исходных данных:', r2_source)
    
    # генерируем случайную квадратную матрицу
    key_matrix = np.random.normal(size=(features.shape[1], features.shape[1]))
    # проверяем обратимость этой матрицы
    try:
        inv_matrix = np.linalg.inv(key_matrix)
        print('Ключевая матрица:')
        display(key_matrix)
    except:
        print('Матрица необратима')
    
    # получим преобразованный набор
    features_new = features @ key_matrix
    print('Преобразованный набор данных:')
    display(features_new.head())
    
    # обучим модель и рассчитаем метрику r_2
    model = LinearRegression().fit(features_new, target)
    predictions = model.predict(features_new)
    r2_new = r2_score(target, predictions)
    print('Метрика R2 модели:', r2_new)
    
    # вернём исходный набор данных, округляя до целых - как это было в исходном датасете
    inv_matrix = np.linalg.inv(key_matrix)
    features_origin = (features_new @ inv_matrix)
    print('Восстановленный набор данных:')
    display(features_origin.head())
    
    # округлим до целых - как это было в исходном датасете
    features_origin = features_origin.astype(int).round()
    features_origin = pd.DataFrame(features_origin)
    data_origin = features_origin.merge(target, left_index=True, 
                                    right_index=True)
    data_origin.columns = data.columns
    
    # дополним его столбцом с предсказаниями
    predictions = pd.DataFrame(predictions)
    data_origin['Предсказания'] = predictions.round()
    
    #сравним метрики
    if round(r2_source, 4) == round(r2_new, 4):
        print('Метрики R2 равны!')
    else:
        print('Метрики R2 не равны!')
   
    return data_origin.head()

Запустим функцию на предоставленных данных.

In [9]:
encryption(df, 'Страховые выплаты')

Метрика R2 на исходных данных: 0.4302010044852066
Ключевая матрица:


array([[-1.36669985, -2.19526805, -0.82980165, -0.53108723],
       [-0.1962677 , -0.54654752, -1.42402406,  0.02002391],
       [ 1.55266218, -2.62815065,  0.985485  , -0.56535184],
       [ 0.58624901, -1.7827832 ,  0.37431442,  0.48606589]])

Преобразованный набор данных:


,0,1,2,3
0,77003.216772,-130382.658599,48821.215527,-28040.675291
1,58992.720829,-99896.648562,37383.299208,-21481.962744
2,32600.214047,-55207.013469,20653.888303,-11871.807941
3,64743.063842,-109608.925051,41065.568624,-23573.779082
4,40517.620740,-68612.230490,25680.456025,-14755.653434


Метрика R2 модели: 0.4302010044852077
Восстановленный набор данных:


,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-2.006487e-12,46.0,38000.0,1.000000e+00
2,-1.050005e-12,29.0,21000.0,2.015259e-12
3,-5.445863e-12,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,1.773187e-12


Метрики R2 равны!


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты,Предсказания
0,0,40,49599,1,0,1.0
1,0,45,37999,1,1,1.0
2,0,29,21000,0,0,0.0
3,0,21,41700,1,0,-0.0
4,1,27,26099,0,0,0.0


### Выводы

**Метрика качества предсказаний одинакова** - как и следовало ожидать в соответствии с теоретическим обоснованием, основанным на способе работы линейной регрессии, и смыслом матричных операций. 

Одним из способов шифрования данных является умножение датасета на обратимую матрицу. Восстановить данные, не зная матрицу, будет сложно. При этом те, у кого есть ключевая матрица, всегда смогут восстановить исходный датасет. **Преобразования прошли успешно** - при округлении данных в восстановленном датасете. Без округления значения несколько отличаются от первоначальных (по всей видимости, из-за особенностей вычислений с типом float64).